In [ ]:
#!git clone https://github.com/zhouhaoyi/Informer2020.git
#!git clone https://github.com/zhouhaoyi/ETDataset.git
# modify the code based on my dataset

In [ ]:
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install scikit_learn
# !{sys.executable} -m pip install torch
# !{sys.executable} -m pip install seaborn

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from argparse import Namespace
import re
from sklearn.metrics import (
    mean_absolute_error,
    root_mean_squared_error,
    mean_absolute_percentage_error
)

In [2]:
if not 'Informer2020' in sys.path:
    sys.path += ['Informer2020']

from data.data_loader import Dataset_Custom
from utils.metrics import metric

In [3]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())

2.8.0+cu128
12.8
True


In [4]:
num_gpus = torch.cuda.device_count()
print("Number of GPUs available:", num_gpus)

for i in range(num_gpus):
    gpu_name = torch.cuda.get_device_name(i)
    print(f"GPU {i}: {gpu_name}")

Number of GPUs available: 1
GPU 0: NVIDIA A30


## 1. Prepare the data

In [5]:
my_data = pd.read_csv('../GD030A_S.csv')

In [6]:
# Define the recover_timestamp function
def recover_timestamp(data):
    # Combine 'date' and 'time' to form a datetime column
    data['datetime'] = pd.to_datetime(data['date'] + ' ' + data['time'].astype(str) + ':00', format='%Y-%m-%d %H:%M')

    # Set 'datetime' as index
    data = data.set_index('datetime')

    # Create a complete range of timestamps with hourly frequency
    full_time_range = pd.date_range(start=data.index.min(), end=data.index.max(), freq='H')

    # Reindex the data to include all timestamps, filling missing rows with NaN
    data_full = data.reindex(full_time_range)

    return data_full

Custom data (xxx.csv) has to include at least 2 features: date(format: YYYY-MM-DD hh:mm:ss) and target feature.

In [ ]:
traffic_full = recover_timestamp(my_data)
traffic_full.drop(['date', 'time'], axis=1, inplace=True)
traffic_full = traffic_full.reset_index(names='date')
traffic_full['date'] = traffic_full['date'].astype(str)
traffic_full

In [ ]:
#traffic_full.to_csv('traffic_full.csv', index=False)

In [60]:
# after covid data
traffic_full = pd.read_csv('../traffic_full.csv')
traffic_full
traffic_after_covid = traffic_full.loc[traffic_full[traffic_full['date'] == '2022-06-03 00:00:00'].index[0]:]
traffic_after_covid = traffic_after_covid.reset_index(drop=True)

In [59]:
#traffic_after_covid.to_csv('../traffic_after_covid.csv', index=False)

## 2. Run training directly using bash

In [5]:
# ensure the shell commands below using the current Python environment
os.environ["PATH"] = f"{sys.prefix}/bin:" + os.environ["PATH"]

In [6]:
cd Informer2020

/home/yl2672496l/Yue/code/transformer/Informer2020


In [7]:
#test
# !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len 24 --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 1 #--inverse

In [27]:
seq_lens = [24 * i for i in range(1, 22)]
print(seq_lens)

[24, 48, 72, 96, 120, 144, 168, 192, 216, 240, 264, 288, 312, 336, 360, 384, 408, 432, 456, 480, 504]


### Train and test the model with different label_len
results1 with lable_len == 24, results with label_len == max(pred_len, seq_len // 3)

#### Entire dataset

In [ ]:
for seq_len in seq_lens:
    print(f"Running seq_len={seq_len}")
    !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 10

# for seq_len in seq_lens:
#     # Calculate label_len: max(pred_len, seq_len // 3)
#     label_len = max(6, seq_len // 3)
    
#     print(f"Running seq_len={seq_len}, label_len={label_len}, pred_len=6")
#     !python -u main_informer.py --model informer --data traffic_full --root_path ../ --features S --seq_len {seq_len} --label_len {label_len} --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 10

Running seq_len=24
Args in experiment:
Namespace(model='informer', data='traffic_full', root_path='../', data_path='traffic_full.csv', features='S', target='flow', freq='h', checkpoints='./checkpoints/', seq_len=24, label_len=24, pred_len=6, enc_in=1, dec_in=1, c_out=1, d_model=512, n_heads=8, e_layers=2, d_layers=1, s_layers=[3, 2, 1], d_ff=2048, factor=5, padding=0, distil=True, dropout=0.05, attn='prob', embed='timeF', activation='gelu', output_attention=False, do_predict=False, mix=True, cols=None, num_workers=0, itr=10, train_epochs=10, batch_size=32, patience=3, learning_rate=0.0001, des='Exp', loss='mse', lradj='type1', use_amp=False, inverse=False, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', detail_freq='h')
Use GPU: cuda:0
>>>>>>>start training : informer_traffic_full_ftS_sl24_ll24_pl6_dm512_nh8_el2_dl1_df2048_atprob_fc5_ebtimeF_dtTrue_mxTrue_Exp_0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 21009
[DEBUG] Original dataset length: 21009
[DEBUG] Cleaned dataset length (NaN 

#### After covid dataset

In [ ]:
for seq_len in seq_lens:
    print(f"Running seq_len={seq_len}")
    !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len {seq_len} --label_len 24 --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 10

# for seq_len in seq_lens:
#     # Calculate label_len: max(pred_len, seq_len // 3)
#     label_len = max(6, seq_len // 3)
    
#     print(f"Running seq_len={seq_len}, label_len={label_len}, pred_len=6")
#     !python -u main_informer.py --model informer --data traffic_after_covid --root_path ../ --features S --seq_len {seq_len} --label_len {label_len} --pred_len 6 --e_layers 2 --d_layers 1 --attn prob --des 'Exp' --itr 10 --train_epochs 10

In [34]:
data = np.load("./results/running_time_traffic_full_24.npy")
data

array([13.86369028,  1.45002213])

## 3. Calculate the metrics

In [11]:
# Extract seq_len and run number from directory name
# Example:
# informer_traffic_full_ftS_sl24_ll24_pl6_dm512_..._Exp_3
# seq_len = 24, run_num = 3
def extract_info(dirname):
    seq_match = re.search(r"sl(\d+)", dirname)
    run_match = re.search(r"Exp_(\d+)", dirname)

    seq_len = int(seq_match.group(1)) if seq_match else None
    run_num = int(run_match.group(1)) + 1 if run_match else None  # Exp_0 → run 1

    return seq_len, run_num

# Compute metrics for each of 6 prediction steps
# pred, true shape: (N, 6, 1)
def compute_step_metrics(pred, true):
    steps = pred.shape[1]
    rows = []

    for step in range(steps):
        p = pred[:, step, 0]
        g = true[:, step, 0]

        mae = mean_absolute_error(g, p)
        rmse = root_mean_squared_error(g, p)
        mape = mean_absolute_percentage_error(g, p) * 100  # convert to percentage
        rows.append((step + 1, mae, rmse, mape))
        
    return rows

#### Metrics of each run and each step (6×10×21)

In [12]:
def whole_metrics(target_dataset):
    all_rows = []
    
    # your result directories (you can replace this with auto-search)
    result_dirs = sorted([
        d for d in os.listdir("results")
        if os.path.isdir(os.path.join("results", d)) and f"informer_{target_dataset}_" in d  # <<< key filtering step
    ])
    
    for d in result_dirs:
        seq_len, run_num = extract_info(d)
    
        pred_path = os.path.join("results", d, "pred_inverse.npy")
        true_path = os.path.join("results", d, "true_inverse.npy")
    
        if not (os.path.exists(pred_path) and os.path.exists(true_path)):
            print(f"Skipping missing: {d}")
            continue
    
        pred = np.load(pred_path)
        true = np.load(true_path)
    
        metrics = compute_step_metrics(pred, true)
    
        for step, mae, rmse, mape in metrics:
            all_rows.append({
                "input_len": seq_len,
                "run_num": run_num,
                "pre_step": step,
                "MAE_test": mae,
                "RMSE_test": rmse,
                "MAPE (%)_test": mape
            })
    
    df = pd.DataFrame(all_rows)
    df = df.sort_values(by=["input_len", "run_num", "pre_step"]).reset_index(drop=True)

    return df

In [30]:
metrics_full_entire_data = whole_metrics("traffic_full")
metrics_full_entire_data
#metrics_full_entire_data.to_csv('../all_metrics_informer_entire_data.csv', index=False)

,input_len,run_num,pre_step,MAE_test,RMSE_test,MAPE (%)_test
0,24,1,1,22.492685,32.477638,28.988224
1,24,1,2,26.168844,38.095703,32.815653
2,24,1,3,27.622231,40.577641,36.270219
3,24,1,4,28.358034,41.717941,38.424060
4,24,1,5,29.342207,43.553413,40.684101
...,...,...,...,...,...,...
1255,504,10,2,28.453787,45.006958,32.844839
1256,504,10,3,30.072334,47.601128,35.301101
1257,504,10,4,31.241228,49.574760,36.256349
1258,504,10,5,32.159279,50.035900,38.303244


In [31]:
metrics_full_after_covid = whole_metrics("traffic_after_covid")
metrics_full_after_covid
#metrics_full_after_covid.to_csv('../all_metrics_informer_after_covid.csv', index=False)

,input_len,run_num,pre_step,MAE_test,RMSE_test,MAPE (%)_test
0,24,1,1,19.113491,25.129717,17.410120
1,24,1,2,23.086666,31.138243,20.457672
2,24,1,3,25.461254,33.751472,22.400559
3,24,1,4,26.946356,35.176872,23.723152
4,24,1,5,26.869194,35.334034,24.328953
...,...,...,...,...,...,...
1255,504,10,2,29.474087,36.802917,26.161003
1256,504,10,3,29.327065,36.681248,25.416061
1257,504,10,4,29.690300,37.326187,26.094425
1258,504,10,5,30.703417,39.161648,27.053139


#### Metrics of each input length (21)

In [32]:
#label_len == max(pred_len, seq_len // 3)
metrics_entire_data = metrics_full_entire_data.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
metrics_entire_data

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.944691,39.958597,33.937828
1,48,27.081740,40.409057,34.893205
2,72,27.446016,40.553551,36.966954
3,96,27.551050,41.393979,35.007789
4,120,27.421511,41.112958,36.939158
5,144,27.713128,41.471491,36.701259
6,168,27.322957,40.987236,36.373228
7,192,27.113147,41.241941,34.820253
8,216,27.703049,41.704202,37.449192
9,240,27.250186,42.097064,35.684882


In [18]:
#lable_len == 24
metrics_entire_data = metrics_full_entire_data.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
metrics_entire_data
#metrics_entire_data.to_csv('../metrics_informer_entire_data.csv', index=False)

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,27.114967,40.347999,33.629718
1,48,27.049395,40.464279,34.209236
2,72,27.309779,40.585444,36.341192
3,96,27.461576,41.215513,35.083345
4,120,27.729210,41.287260,36.931214
5,144,27.851565,41.567416,36.436125
6,168,27.509241,41.199399,36.524670
7,192,27.293735,41.271769,36.138757
8,216,27.203077,41.346274,36.067747
9,240,27.298865,41.822577,36.368179


In [33]:
#label_len == max(pred_len, seq_len // 3)
metrics_after_covid = metrics_full_after_covid.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
metrics_after_covid

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,25.701250,33.466528,23.606928
1,48,26.486588,34.616418,24.487525
2,72,29.804218,38.229649,27.609996
3,96,27.719577,36.004003,26.825592
4,120,31.339854,40.362905,30.836614
5,144,33.107638,41.606084,31.614325
6,168,35.567366,44.348384,33.346493
7,192,34.168946,42.571833,32.390949
8,216,28.951880,36.106669,31.358253
9,240,28.461486,35.926488,31.501973


In [26]:
#lable_len == 24
metrics_after_covid = metrics_full_after_covid.groupby("input_len")[["MAE_test", "RMSE_test", "MAPE (%)_test"]].mean().reset_index()
#metrics_after_covid.to_csv('../metrics_informer_after_covid.csv', index=False)
metrics_after_covid

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.918601,34.784886,24.879766
1,48,27.556540,35.821873,24.921795
2,72,29.252841,37.666548,27.301196
3,96,30.217613,38.683707,28.015250
4,120,30.552287,39.348547,29.846335
5,144,32.854411,41.518079,31.584961
6,168,32.003566,40.285292,31.648953
7,192,33.417555,42.112622,34.222760
8,216,28.220300,35.400657,31.007888
9,240,28.169608,35.686461,31.432002


### Compare the metrics with model built in results

In [22]:
# Collect all result directories
target_dataset = "traffic_after_covid"
result_dirs = sorted([
    d for d in os.listdir("results")
    if os.path.isdir(os.path.join("results", d)) and f"informer_{target_dataset}_" in d  # <<< key filtering step
])

# Dictionary to store metrics per seq_len
metrics_dict = {}

for d in result_dirs:
    folder_path = os.path.join("results", d)
    
    metrics_path = os.path.join(folder_path, "metrics_inverse.npy")
    if not os.path.exists(metrics_path):
        print(f"[WARNING] Missing metrics_inverse.npy in {folder_path}")
        continue
    
    metrics = np.load(metrics_path)  # [MAE, MSE, RMSE, MAPE, MSPE]
    
    # Extract seq_len from folder name (assuming folder name contains 'sl<seq_len>_run<run_num>')
    seq_len = int([s for s in d.split("_") if s.startswith("sl")][0][2:])
    
    if seq_len not in metrics_dict:
        metrics_dict[seq_len] = {"MAE": [], "RMSE": [], "MAPE": []}
    
    metrics_dict[seq_len]["MAE"].append(metrics[0])
    metrics_dict[seq_len]["RMSE"].append(metrics[2])
    metrics_dict[seq_len]["MAPE"].append(metrics[3])

# Calculate mean over all runs per seq_len
rows = []
for seq_len, vals in metrics_dict.items():
    rows.append({
        "input_len": seq_len,
        "MAE_test": np.mean(vals["MAE"]),
        "RMSE_test": np.mean(vals["RMSE"]),
        "MAPE (%)_test": np.mean(vals["MAPE"]) *100
    })

df_inverse_avg = pd.DataFrame(rows).sort_values("input_len").reset_index(drop=True)
df_inverse_avg

,input_len,MAE_test,RMSE_test,MAPE (%)_test
0,24,26.918604,34.910751,24.879768
1,48,27.556540,35.952579,24.921795
2,72,29.252844,37.781456,27.301199
3,96,30.217615,38.881721,28.015249
4,120,30.552286,39.505878,29.846334
5,144,32.854408,41.626030,31.584963
6,168,32.003563,40.360962,31.648952
7,192,33.417553,42.187721,34.222759
8,216,28.220301,35.503487,31.007893
9,240,28.169607,35.782070,31.431999


In [49]:
avg_by_seq.compare(df_inverse_avg)  

MAE_test             RMSE_test            MAPE (%)_test           
         self      other       self      other          self      other
0   26.827424  26.827423  40.091506  40.248596     33.976340  33.976345
1   27.086889  27.086889  40.371117  40.532936     35.322956  35.322960
2   27.484757  27.484756  40.414853  40.567635     37.142969  37.142967
3   27.565523  27.565521  41.038521  41.195618     35.855909  35.855907
4   27.364081  27.364080  41.213944  41.375389     35.366292  35.366291
5   27.528013  27.528015  41.292334  41.451664     35.943623  35.943626
6   27.900328  27.900330  41.459094  41.608902     37.013240  37.013237
7   27.311530  27.311533  41.252090  41.407749     34.981317  34.981316
8   27.348362  27.348362  40.972219  41.117546     37.868599  37.868603
9   27.662873  27.662872  41.741723  41.893860     37.931720  37.931721
10  27.909886  27.909887  42.613484  42.781487     36.033699  36.033703
11  28.428512  28.428513  42.839811  43.016197     38.538594  38.538597
12  27.657373  27.657373  42.875749  43.059753     34.077934  34.077934
13  28.042166  28.042166  42.898795  43.075539     35.931267  35.931267
14  28.378985  28.378986  43.788344  43.971962     34.976826  34.976826
15  28.860072  28.860071  44.196937  44.398430     36.085584  36.085587
16  29.124343  29.124344  44.124789  44.320198     38.010948  38.010944
17  28.937098  28.937099  44.651621  44.846455     36.279675  36.279671
18  29.117829  29.117828  44.748922  44.940174     37.068310  37.068310
19  29.624438  29.624439  46.003536  46.211472     36.347774  36.347771
20  29.368494  29.368494  46.124282  46.341087     36.350152  36.350151

## 4. Calculate the model size

In [ ]:
"""
Informer Model Size Calculator
Based on the original Informer implementation arguments
"""

def calculate_informer_size(
    enc_in=7,           # encoder input size (number of input features)
    dec_in=7,           # decoder input size
    c_out=7,            # output size
    d_model=512,        # dimension of model
    n_heads=8,          # number of attention heads
    e_layers=2,         # number of encoder layers
    d_layers=1,         # number of decoder layers
    d_ff=2048,          # dimension of feedforward network
    seq_len=96,         # input sequence length (doesn't affect params)
    label_len=48,       # start token length (doesn't affect params)
    pred_len=24,        # prediction length (doesn't affect params)
    freq='h',           # frequency for time features
):
    """
    Calculate the approximate number of parameters in Informer model.
    
    Note: seq_len, label_len, and pred_len do NOT affect parameter count,
    only computational cost and memory usage.
    """
    
    total_params = 0
    breakdown = {}
    
    # 1. Input Embedding Layers
    # Encoder: projects enc_in features to d_model
    enc_embedding = enc_in * d_model
    breakdown['Encoder Value Embedding'] = enc_embedding
    total_params += enc_embedding
    
    # Decoder: projects dec_in features to d_model
    dec_embedding = dec_in * d_model
    breakdown['Decoder Value Embedding'] = dec_embedding
    total_params += dec_embedding
    
    # Positional Embedding (learned or fixed - typically small, estimated)
    pos_embedding = d_model * 2  # Approximate for both encoder and decoder
    breakdown['Positional Embeddings'] = pos_embedding
    total_params += pos_embedding
    
    # Time Features Embedding (depends on freq, estimated ~4-10 features)
    time_features = 5  # Approximate number of time features
    time_embedding = time_features * d_model
    breakdown['Time Features Embedding'] = time_embedding
    total_params += time_embedding
    
    # 2. Encoder Layers
    # Each encoder layer has:
    # - Multi-head attention: 4 * d_model * d_model (Q, K, V, Output projections)
    # - Feed-forward network: 2 * d_model * d_ff
    # - Layer normalization: 2 * 2 * d_model (2 LN layers, each with scale and bias)
    
    params_per_encoder = (
        4 * d_model * d_model +  # Attention projections
        2 * d_model * d_ff +      # FFN
        4 * d_model               # Layer norms (2 LN * 2 params)
    )
    encoder_params = e_layers * params_per_encoder
    breakdown['Encoder Layers'] = encoder_params
    breakdown[f'  - Per Encoder Layer'] = params_per_encoder
    total_params += encoder_params
    
    # 3. Decoder Layers
    # Each decoder layer has:
    # - Self-attention: 4 * d_model * d_model
    # - Cross-attention: 4 * d_model * d_model
    # - Feed-forward network: 2 * d_model * d_ff
    # - Layer normalization: 3 * 2 * d_model (3 LN layers)
    
    params_per_decoder = (
        8 * d_model * d_model +   # Self-attention + Cross-attention
        2 * d_model * d_ff +      # FFN
        6 * d_model               # Layer norms (3 LN * 2 params)
    )
    decoder_params = d_layers * params_per_decoder
    breakdown['Decoder Layers'] = decoder_params
    breakdown[f'  - Per Decoder Layer'] = params_per_decoder
    total_params += decoder_params
    
    # 4. Output Projection Layer
    # Projects from d_model to c_out
    output_projection = d_model * c_out + c_out  # weights + bias
    breakdown['Output Projection'] = output_projection
    total_params += output_projection
    
    return total_params, breakdown


# Example calculation with default arguments
if __name__ == "__main__":
    # YOUR DATA CONFIGURATION - Univariate time series
    total_your_data, breakdown_your_data = calculate_informer_size(
        enc_in=1,        # YOUR DATA: single input feature
        dec_in=1,        # YOUR DATA: single decoder input
        c_out=1,         # YOUR DATA: single output
        d_model=512,
        n_heads=8,
        e_layers=2,
        d_layers=1,
        d_ff=2048,
        seq_len=96,      # Note: doesn't affect params
        label_len=48,    # Note: doesn't affect params
        pred_len=24,     # Note: doesn't affect params
    )
    
    print("=" * 60)
    print("INFORMER MODEL SIZE - YOUR DATA (UNIVARIATE)")
    print("=" * 60)
    print("\nYour Configuration:")
    print(f"  enc_in (input features): 1  ← YOUR DATA")
    print(f"  dec_in: 1  ← YOUR DATA")
    print(f"  c_out: 1  ← YOUR DATA")
    print(f"  d_model: 512")
    print(f"  n_heads: 8")
    print(f"  e_layers (encoder): 2")
    print(f"  d_layers (decoder): 1")
    print(f"  d_ff: 2048")
    print(f"\n  seq_len: 96 (doesn't affect params)")
    print(f"  label_len: 48 (doesn't affect params)")
    print(f"  pred_len: 24 (doesn't affect params)")
    
    print("\n" + "=" * 60)
    print("Parameter Breakdown:")
    print("=" * 60)
    for key, value in breakdown_your_data.items():
        print(f"{key:.<50} {value:>10,}")
    
    print("=" * 60)
    print(f"{'TOTAL PARAMETERS':.<50} {total_your_data:>10,}")
    print(f"{'MODEL SIZE':.<50} {total_your_data/1e6:>9.2f}M")
    print("=" * 60)
    
    # Also calculate default multivariate for comparison
    total_default, breakdown_default = calculate_informer_size(
        enc_in=7,
        dec_in=7,
        c_out=7,
        d_model=512,
        n_heads=8,
        e_layers=2,
        d_layers=1,
        d_ff=2048,
        seq_len=96,
        label_len=48,
        pred_len=24,
    )
    
    print("\n" + "=" * 60)
    print("COMPARISON: YOUR DATA vs DEFAULT MULTIVARIATE")
    print("=" * 60)
    print(f"{'Your Data (enc_in=1)':.<40} {total_your_data/1e6:>8.2f}M")
    print(f"{'Default (enc_in=7)':.<40} {total_default/1e6:>8.2f}M")
    print(f"{'Difference':.<40} {(total_default-total_your_data)/1e6:>8.2f}M")
    print(f"{'Percentage smaller':.<40} {(1-total_your_data/total_default)*100:>7.1f}%")
    print("=" * 60)
    
    print("\n" + "=" * 60)
    print("PARAMETER BREAKDOWN:")
    print("=" * 60)
    for key, value in breakdown.items():
        print(f"{key:.<50} {value:>10,}")
    
    print("=" * 60)
    print(f"{'TOTAL PARAMETERS':.<50} {total:>10,}")
    print(f"{'MODEL SIZE':.<50} {total/1e6:>9.2f}M")
    print("=" * 60)
    
    print("\n" + "=" * 60)
    print("IMPORTANT NOTES:")
    print("=" * 60)
    print("✓ Parameters affected by: enc_in, dec_in, c_out, d_model,")
    print("  n_heads, e_layers, d_layers, d_ff")
    print("\n✗ Parameters NOT affected by: seq_len, label_len, pred_len")
    print("  (these only affect memory usage and computation time)")
    print("=" * 60)
    
    # Additional examples
    print("\n" + "=" * 60)
    print("COMPARISON WITH DIFFERENT CONFIGURATIONS:")
    print("=" * 60)
    
    configs = [
        {"name": "Small", "d_model": 256, "e_layers": 2, "d_layers": 1, "d_ff": 1024},
        {"name": "Default", "d_model": 512, "e_layers": 2, "d_layers": 1, "d_ff": 2048},
        {"name": "Large", "d_model": 768, "e_layers": 4, "d_layers": 2, "d_ff": 3072},
    ]
    
    for config in configs:
        total, _ = calculate_informer_size(
            d_model=config["d_model"],
            e_layers=config["e_layers"],
            d_layers=config["d_layers"],
            d_ff=config["d_ff"]
        )
        print(f"{config['name']:.<20} {total/1e6:>8.2f}M parameters")
    print("=" * 60)

## 5. Calculate the running time

In [36]:
def combine_npy_by_prefix(directory, prefix):
    """
    Scans the given directory for files matching:
        <prefix>_<seq_len>.npy
    Loads each file (expected to contain [training_time, inference_time]),
    and writes a CSV named <prefix>.csv
    """
    # regex: prefix_seqLen.npy
    # escaping prefix just in case it contains regex characters
    pattern = re.compile(r"(" + re.escape(prefix) + r")_(\d+)\.npy")

    records = []

    for fname in os.listdir(directory):
        match = pattern.match(fname)
        if not match:
            continue

        seq_len = int(match.group(2))
        arr = np.load(os.path.join(directory, fname))

        if len(arr) < 2:
            raise ValueError(f"File {fname} does not contain at least two values.")

        training_time, inference_time = arr[0], arr[1]
        records.append((seq_len, training_time, inference_time))

    if not records:
        raise ValueError("No matching files found for prefix: " + prefix)

    df = pd.DataFrame(records, columns=["seq_len", "training_time", "inference_time"])
    df = df.sort_values("seq_len")

    # out_csv = os.path.join(directory, prefix + ".csv")
    # df.to_csv(out_csv, index=False)
    return df

In [43]:
running_time_entire_data = combine_npy_by_prefix("./results", "running_time_traffic_full")
running_time_entire_data
# running_time_entire_data.to_csv('../running_time_informer_entire_data.csv', index=False)

In [44]:
running_time_after_covid = combine_npy_by_prefix("./results", "running_time_traffic_after_covid")
# running_time_after_covid.to_csv('../running_time_informer_after_covid.csv', index=False)